# Training the CNN for temperature prediction


In [ ]:

import os
from CNN import SequenceCNN
from torch.utils.data import DataLoader as TorchDataLoader

In [2]:
import torch
from torch.optim import lr_scheduler

import torch.optim as optim
import torch.nn as nn
from TemperatureDataset import TemperatureDataset
from functools import partial
import os
import tempfile
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms

In [3]:
dataset = TemperatureDataset('data_temp_max', 'data_temp_min')

100%|██████████| 562/562 [00:38<00:00, 14.47it/s]


In [4]:
import optuna
from tqdm import tqdm

def objective(trial):
    # Remove activations_map from here, since it is already defined and used in cell 8.
    # Also, do not redefine batch_size or train_loader, as they are already defined globally.
    # You can remove this placeholder entirely.

    # Use strings to represent tuples for Optuna compatibility
    hidden_dims_options = ["32,32", "32,64", "64,64", "64,32", "32,128", "128,32","64,128","128,64", "128,128", "64,128", "32,256", "128,256", "256,256"]
    kernel_sizes_options = ["3,3", "5,3", "3,5", "5,5"]
    # Define activations_options outside the objective function to avoid dynamic value space error
    activations_options = ["relu,relu", "tanh,relu", "leaky_relu,sigmoid", "sigmoid,tanh", "relu,sigmoid", "tanh,tanh", "leaky_relu,leaky_relu", "tanh,leaky_relu", "relu,leaky_relu", "sigmoid,sigmoid", "sigmoid,leaky_relu", "relu,tanh", "tanh,relu", "leaky_relu,relu", "selu,leaky_relu"]
    hidden_dims_str = trial.suggest_categorical("hidden_dims", hidden_dims_options)
    kernel_sizes_str = trial.suggest_categorical("kernel_sizes", kernel_sizes_options)
    activations_str = trial.suggest_categorical("activations", activations_options)

    hidden_dims = tuple(map(int, hidden_dims_str.split(',')))
    kernel_sizes = tuple(map(int, kernel_sizes_str.split(',')))
    activations = tuple(activations_str.split(','))

    lr = trial.suggest_loguniform("lr", 1e-4, 1e-2)
    batch_size = trial.suggest_categorical("batch_size", (16, 32, 64))
    num_epochs = 5
    sample_input, sample_target = dataset[0]
    input_dim = sample_input.shape[-1]
    output_dim = sample_target.shape[-1]


    model = SequenceCNN(
        input_dim=input_dim,
        hidden_dims=list(hidden_dims),
        kernel_sizes=list(kernel_sizes),
        output_dim=output_dim,
        activations=activations
    )
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    train_loader = TorchDataLoader(dataset, batch_size=batch_size, shuffle=True)

    for epoch in range(num_epochs):
        print(f"Starting training epoch {epoch + 1}/{num_epochs}")
        print(f"Learning rate: {lr}, Batch size: {batch_size}, Hidden dims: {hidden_dims}, Kernel sizes: {kernel_sizes}, Activations: {activations}")
        model.train()
        for batch in tqdm(train_loader):
            inputs, targets = batch
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

    return loss.item()

# Example usage:
# study = optuna.create_study(direction="minimize")
# study.optimize(objective, n_trials=10)

c:\Users\henri\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Split the dataset into train, validation, and test sets
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create DataLoaders for each set
batch_size = 32  # You can adjust this as needed

train_loader = TorchDataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = TorchDataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = TorchDataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [6]:
import torch
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA device count:", torch.cuda.device_count())
    print("Current device:", torch.cuda.current_device())
    print("Device name:", torch.cuda.get_device_name(torch.cuda.current_device()))
else:
    print("Using CPU")

CUDA available: True
CUDA device count: 1
Current device: 0
Device name: NVIDIA GeForce RTX 3060 Laptop GPU


In [ ]:
import optuna
import warnings
warnings.filterwarnings("ignore")
# Run Optuna hyperparameter search
study = optuna.create_study(
    study_name="CNN_Sequence_Optimization_2",
    storage="sqlite:///optuna_study.db",
    load_if_exists=True,
    direction="minimize",
)

study.optimize(objective, n_trials=20)

# Get the best config
best_params = study.best_params
print(f"Best params: {best_params}")


[I 2025-06-07 15:21:18,513] A new study created in RDB with name: CNN_Sequence_Optimization_2


Starting training epoch 1/5
Learning rate: 0.0005588459736553247, Batch size: 64, Hidden dims: (256, 256), Kernel sizes: (5, 3), Activations: ('sigmoid', 'tanh')


100%|██████████| 57516/57516 [03:02<00:00, 315.11it/s]


Starting training epoch 2/5
Learning rate: 0.0005588459736553247, Batch size: 64, Hidden dims: (256, 256), Kernel sizes: (5, 3), Activations: ('sigmoid', 'tanh')


100%|██████████| 57516/57516 [03:03<00:00, 312.76it/s]


...

100%|██████████| 115031/115031 [06:37<00:00, 289.17it/s]


Starting training epoch 5/5
Learning rate: 0.0008911494812012466, Batch size: 32, Hidden dims: (64, 128), Kernel sizes: (3, 5), Activations: ('relu', 'sigmoid')


100%|██████████| 115031/115031 [06:40<00:00, 287.41it/s]
[I 2025-06-08 04:02:57,694] Trial 19 finished with value: 0.03344499692320824 and parameters: {'hidden_dims': '64,128', 'kernel_sizes': '3,5', 'activations': 'relu,sigmoid', 'lr': 0.0008911494812012466, 'batch_size': 32}. Best is trial 10 with value: 0.018964406102895737.


Best params: {'hidden_dims': '32,64', 'kernel_sizes': '5,5', 'activations': 'leaky_relu,relu', 'lr': 0.002507800985448251, 'batch_size': 32}


KeyError: 'l'

In [8]:
activations_map = {
    'relu': torch.nn.ReLU(),
    'tanh': torch.nn.Tanh(),
    'sigmoid': torch.nn.Sigmoid(),
    'leaky_relu': torch.nn.LeakyReLU(),
    'elu': torch.nn.ELU(),
    'gelu': torch.nn.GELU(),
    'selu': torch.nn.SELU(),
    'none': nn.Identity()
}
# Prepare activations
activations = [activations_map[act] for act in best_params['activations'].split(',')]

# Build and train the best model on the full training set
sample_input, sample_target = dataset[0]
input_dim = sample_input.shape[-1]
output_dim = sample_target.shape[-1]
best_model = SequenceCNN(
    input_dim=input_dim,
    hidden_dims=list(best_params['hidden_dims']),
    kernel_sizes=list(best_params['kernel_sizes']),
    output_dim=output_dim,
    activations=activations
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
best_model.to(device)

optimizer = optim.Adam(best_model.parameters(), lr=best_params['lr'])
criterion = nn.MSELoss()

# Use the full train_loader for training
num_epochs = 5
for epoch in range(num_epochs):
    best_model.train()
    for batch in train_loader:
        inputs, targets = batch
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = best_model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

# Evaluate on the test set
best_model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = best_model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item() * inputs.size(0)
test_loss /= len(test_loader.dataset)
print(f"Test loss: {test_loss}")


AssertionError: hidden_dims and kernel_sizes must have the same length

In [ ]:
# Save the best model
model_path = Path("best_cnn_model.pth")
torch.save(best_model.state_dict(), model_path)
print(f"Best model saved to {model_path}")
